In [1]:
%reload_kedro

2019-12-07 21:41:35,475 - root - INFO - ** Kedro project sneakairs
2019-12-07 21:41:35,476 - root - INFO - Defined global variable context


In [2]:
import pandas as pd
from pathlib import Path

In [3]:
adidas = context.io.load('01_adidas_silhouette').split(sep='\n')
asics = context.io.load('01_asics_silhouette').split(sep='\n')
converse = context.io.load('01_converse_silhouette').split(sep='\n')
jordan = context.io.load('01_jordan_silhouette').split(sep='\n')
newbalance = context.io.load('01_newbalance_silhouette').split(sep='\n')
nike = context.io.load('01_nike_silhouette').split(sep='\n')
puma = context.io.load('01_puma_silhouette').split(sep='\n')
reebok = context.io.load('01_reebok_silhouette').split(sep='\n')
vans = context.io.load('01_vans_silhouette').split(sep='\n')

2019-12-07 21:41:35,516 - kedro.io.data_catalog - INFO - Loading data from `01_adidas_silhouette` (TextLocalDataSet)...
2019-12-07 21:41:35,531 - kedro.io.data_catalog - INFO - Loading data from `01_asics_silhouette` (TextLocalDataSet)...
2019-12-07 21:41:35,551 - kedro.io.data_catalog - INFO - Loading data from `01_converse_silhouette` (TextLocalDataSet)...
2019-12-07 21:41:35,568 - kedro.io.data_catalog - INFO - Loading data from `01_jordan_silhouette` (TextLocalDataSet)...
2019-12-07 21:41:35,588 - kedro.io.data_catalog - INFO - Loading data from `01_newbalance_silhouette` (TextLocalDataSet)...
2019-12-07 21:41:35,608 - kedro.io.data_catalog - INFO - Loading data from `01_nike_silhouette` (TextLocalDataSet)...
2019-12-07 21:41:35,630 - kedro.io.data_catalog - INFO - Loading data from `01_puma_silhouette` (TextLocalDataSet)...
2019-12-07 21:41:35,648 - kedro.io.data_catalog - INFO - Loading data from `01_reebok_silhouette` (TextLocalDataSet)...
2019-12-07 21:41:35,662 - kedro.io.data

In [4]:
text_files = [adidas, asics, converse, jordan, newbalance, nike, puma, reebok, vans]
brands = ['adidas', 'asics', 'converse', 'jordan', 'new-balance', 'nike', 'puma', 'reebok', 'vans']

In [5]:
def txt_2_dict(x):
    silhouette_dict = {}
    current_key = None
    for i in range(len(x)):
        try:
            silhouette = eval(x[i])
        except:
            silhouette = x[i]
        if isinstance(silhouette, list):
            current_key = silhouette[0]
            silhouette_dict[current_key] = []
        else:
            silhouette_dict[current_key].append(silhouette)
    return silhouette_dict

In [6]:
def remove_duplicates(x):
    for key in x:
        silhouettes = x[key]
        silhouettes = list(set(silhouettes))
        x[key] = silhouettes
    return x

In [7]:
def dict_2_df(dictionary, brand):    
    df = pd.DataFrame(data=list(dictionary.keys()), columns=['sneaker_line'])
    df['silhouettes'] = df['sneaker_line'].apply(lambda x: dictionary[x])
    df = df.explode('silhouettes')
    df['brand'] = brand
    df = df.reset_index(drop=True)
    return df

In [8]:
for i in range(len(text_files)):
    text = text_files[i]
    brand = brands[i]
    
    output = txt_2_dict(text)
    output = remove_duplicates(output)
    output = dict_2_df(output, brand)
    
    file_path = 'data/02_intermediate/' + brand + '-silhouette.csv'
    output.to_csv(Path(file_path), index=False)

In [9]:
for i in range(len(text_files)):
    text = text_files[i]
    brand = brands[i]
    
    output = txt_2_dict(text)
    output = remove_duplicates(output)
    
    file_path = 'data/02_intermediate/' + brand + '-silhouette.txt'
    file = open(Path(file_path),"w+")
    file.write(str(output))
    file.close()

In [10]:
all_silhouettes = []
for i in range(len(text_files)):
    text = text_files[i]    
    output = txt_2_dict(text)
    output = remove_duplicates(output)
    output = list(output.values())
    for j in range(len(output)):
        all_silhouettes.extend(output[j])
all_silhouettes = list(set(all_silhouettes))
all_silhouettes.remove('')
file_path = 'data/02_intermediate/all-silhouettes.txt'
file = open(Path(file_path),"w+")
file.write(str(all_silhouettes))
file.close()